In [3]:
!pip install pdfminer.six
!pip install pyPdf
!pip install PyPDF2
!pip install pymupdf

In [50]:
from google.colab import drive
import fitz
import pandas as pd
import numpy as np
import re
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/HKESS_Scraping')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Convert Pdf to html

In [15]:
for f in os.listdir():
  if f.endswith('.pdf'):
    doc = fitz.open(f) 
    with open(f'{f}.html','w') as f:
      htmls = [page.getText('html') for page in doc]
      htmlString = ''.join(htmls)
      f.write(htmlString)

# Parsing


In [88]:
from bs4 import BeautifulSoup


htmlFiles = [f for f in os.listdir() if f.endswith('.html')]

htmlfile = htmlFiles[0]

with open(htmlfile,'r') as f:
  htmlString = f.read()

soup = BeautifulSoup(htmlString)

In [101]:
mths = f"({'|'.join(['January','February','March','April','May','June','July','August','September','October','November','December',])})"
blacklist_patterns = [
  '以下所列之僱主均為',
  'denotesnewlyapprovedapplication',
  'belowlistedemployersareseparateapplicant',
  'Remark',
  'Employername',
  '僱主名稱',
  'Wagesubsidyamount',
  mths+'(\d{4})',
  '工資補貼金額',
  '\d{4}年\d{1,2}月至\d{4}年',
  'Committedheadcount',
  'underpayroll',
  '承諾受薪僱員人數'
]
unified_bl_pat = '|'.join([f'({p})' for p in blacklist_patterns])
unified_bl_pat

'(以下所列之僱主均為)|(denotesnewlyapprovedapplication)|(belowlistedemployersareseparateapplicant)|(Remark)|(Employername)|(僱主名稱)|(Wagesubsidyamount)|((January|February|March|April|May|June|July|August|September|October|November|December)(\\d{4}))|(工資補貼金額)|(\\d{4}年\\d{1,2}月至\\d{4}年)|(Committedheadcount)|(underpayroll)|(承諾受薪僱員人數)'

In [40]:
divpages = soup.findAll('div')

In [103]:
df.head(12)

,html,plain_txt,plain_txt_striped,plain_txt_nopunc,plain_txt_nopunc_noalpganum,is_bl
0,"[[以下所列之僱主均為個別之「保就業」計劃申請者，並為其相應之僱員開設獨立之強積金計劃], ...",以下所列之僱主均為個別之「保就業」計劃申請者，並為其相應之僱員開設獨立之強積金計劃 / 獲強...,以下所列之僱主均為個別之「保就業」計劃申請者，並為其相應之僱員開設獨立之強積金計劃 / 獲強...,以下所列之僱主均為個別之保就業計劃申請者並為其相應之僱員開設獨立之強積金計劃獲強積金豁免職業...,以下所列之僱主均為個別之保就業計劃申請者並為其相應之僱員開設獨立之強積金計劃獲強積金豁免職業...,True
1,[[[* denotes newly approved application / * ]]...,* denotes newly approved application / * 為新獲批之申請,* denotes newly approved application / * 為新獲批之申請,denotesnewlyapprovedapplication為新獲批之申請,為新獲批之申請,True
2,[[Remark]],Remark,Remark,Remark,,True
3,"[[Employer name (English) / ], [ 僱主名稱], [ (], ...",Employer name (English) / 僱主名稱 (英文),Employer name (English) / 僱主名稱 (英文),EmployernameEnglish僱主名稱英文,僱主名稱英文,True
4,"[[Employer name (Chinese) / ], [ 僱主名稱], [ (], ...",Employer name (Chinese) / 僱主名稱 (中文),Employer name (Chinese) / 僱主名稱 (中文),EmployernameChinese僱主名稱中文,僱主名稱中文,True
5,[[ Wage subsidy amount (Jun ]],Wage subsidy amount (Jun,Wage subsidy amount (Jun,WagesubsidyamountJun,,True
6,[[2020 - August 2020) / ]],2020 - August 2020) /,2020 - August 2020) /,2020August2020,,True
7,"[[工資補貼金額], [ ]]",工資補貼金額,工資補貼金額,工資補貼金額,工資補貼金額,True
8,"[[(2020], [年], [6], [月至], [2020], [年], [8], [月...",(2020年6月至2020年8月),(2020年6月至2020年8月),2020年6月至2020年8月,年月至年月,True
9,[[ Committed headcount ]],Committed headcount,Committed headcount,Committedheadcount,,True


In [109]:
divpage = divpages[-2]
ps = divpage.findAll('p')

df = pd.DataFrame({'html':ps})

df['plain_txt'] = df.html.apply(lambda x: x.text)
df['plain_txt_striped'] = df.plain_txt.apply(lambda x: x.strip())
df['plain_txt_nopunc'] = df.plain_txt_striped.apply(lambda x: re.sub('\W','',x))
df['plain_txt_nopunc_noalpganum'] = df.plain_txt_nopunc.apply(lambda x: re.sub('[A-Za-z0-9]','',x))
df['is_bl'] = df.plain_txt_nopunc.apply(lambda x: len(re.findall(unified_bl_pat,x,re.I))> 0)

df = df[df.is_bl == False]

df['is_original_numeric'] = df.plain_txt_striped.apply(lambda x: x.isnumeric())
df['is_after_numeric'] = df.plain_txt_striped.apply(lambda x: re.sub('\W','',x).isnumeric() )
df['is_star'] = (df.plain_txt_striped == '*')

def estimate_role(row):
  if len(row.plain_txt_striped) == 0:
    return 'garbage'
  if row.is_star:
    return 'remark'
  if row.is_original_numeric:
    return 'headcount'
  if row.is_after_numeric:
    return 'amt'
  if len(row.plain_txt_nopunc_noalpganum) > 0:
    return 'ch_name'
  return 'en_name'

df['estimated_role'] = df.apply(estimate_role,axis=1)
df = df[df.estimated_role != 'garbage']
df['estimated_role_lag'] = df.estimated_role.shift(1)
df = df[df.estimated_role != 'remark']
df['grp_id'] = (df.estimated_role == 'en_name').cumsum()

df2 = df.pivot_table(index='grp_id',columns='estimated_role',aggfunc={'plain_txt':lambda x: x.iloc[0]}).plain_txt
df2.reset_index(drop=True,inplace=True)

In [111]:
df2.head(50)

estimated_role,amt,ch_name,en_name,headcount
0,"135,528.00",殿堂公共關係顧問集團有限公司,DT COMMUNICATIONS ASIA PACIFIC LIMITED,7
1,"593,190.00",NaN,DT LOGISTICS (HONG KONG) LIMITED,23
2,"153,000.00",大田証劵期貨有限公司,DT SECURITIES & FUTURES CO LTD,6
3,"212,574.00",曉思會計及秘書服務有限公司,DTC ACCOUNTING & SECRETARY SERVICE LIMITED,11
4,"27,000.00",廸泰工程貿易有限公司,DTC ENGINEERING & TRADING COMPANY LIMITED,1
5,"126,750.00",廸泰工程有限公司,"DTC ENGINEERING CO., LIMITED",5
6,"238,500.00",鼎天資產管理(香港)有限公司,DTC INVESTMENT MANAGEMENT (HK) LIMITED,7
7,"49,500.00",協成紡織有限公司,DTC TEXTILES LIMITED,2
8,"54,000.00",傲硏資訊科技有限公司,DTG NEO SCIENTIFIC LIMITED,2
9,"108,000.00",NaN,DTI ASIA PACIFIC LIMITED,4
